In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
g = 9.81


vec_angulos = [$\phi$, $\psi$, $p_\phi$, $p_\psi$]

In [ ]:
def function (vec_angulos, t):
    div = 2-np.cos(vec_angulos[0]-vec_angulos[1])**2    
    pphi_dot = -(2*(vec_angulos[2]*np.cos(vec_angulos[0]-vec_angulos[1])-2*vec_angulos[3])*(vec_angulos[3]*np.cos(vec_angulos[0]-vec_angulos[1]))*np.sin(vec_angulos[0]-vec_angulos[1]))/div**2 - 2*g*np.sin(vec_angulos[0])
    ppsi_dot = -(2*(vec_angulos[2]*np.cos(vec_angulos[1]-vec_angulos[0])-2*vec_angulos[3])*(vec_angulos[3]*np.cos(vec_angulos[1]-vec_angulos[0]))*np.sin(vec_angulos[1]-vec_angulos[0]))/div**2 - g*np.sin(vec_angulos[1]) 
    phi_dot = (vec_angulos[2]-vec_angulos[3]*np.cos(vec_angulos[1]-vec_angulos[0]))/div
    psi_dot = (2*vec_angulos[3]-vec_angulos[2]*np.cos(vec_angulos[1]-vec_angulos[0]))/div

    return np.array([phi_dot, psi_dot, pphi_dot, ppsi_dot])


In [ ]:
def Runge_Kutta (vec_angulos, h, t):
    k_1 = h*function(vec_angulos, t)
    k_2 = h*function(vec_angulos + k_1/2, t + h/2)
    k_3 = h*function(vec_angulos + k_2/2, t + h/2)
    k_4 = h*function(vec_angulos + k_3, t + h)

    return vec_angulos + (k_1 + k_2 + k_3 + k_4)/6